In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import tensorflow as tf
import random
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
# Set the path to the dataset directory
dataset_path = "sort_sports/"

# Set the image dimensions and batch size
img_height, img_width = 150, 150
batch_size = 32

In [ ]:
# Data preprocessing and augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 0 images belonging to 1 classes.
Found 0 images belonging to 1 classes.


In [ ]:
# Build and train the CNN model
model_cnn = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(5, activation='softmax')
])

model_cnn.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

history = model_cnn.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10
)

ValueError: ignored

In [ ]:
# Get the training and validation accuracy values from the history object
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

# Plot the training and validation accuracy curves
epochs = range(1, len(train_accuracy) + 1)

plt.plot(epochs, train_accuracy, 'b', label='Training Accuracy')
plt.plot(epochs, val_accuracy, 'r', label='Validation Accuracy')

plt.title('Training and Validation Accuracy for CNN')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

In [ ]:
# Apply transfer learning using pre-trained model
base_model = tf.keras.applications.MobileNetV2(input_shape=(img_height, img_width, 3),
                                               include_top=False,
                                               weights='imagenet')

base_model.trainable = False

model_transfer_learning = keras.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dense(5, activation='softmax')
])

model_transfer_learning.compile(optimizer='adam',
                                loss='categorical_crossentropy',
                                metrics=['accuracy'])

history_tf = model_transfer_learning.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10
)


In [ ]:
# Get the training and validation accuracy values from the history object
train_accuracy = history_tf.history['accuracy']
val_accuracy = history_tf.history['val_accuracy']

# Plot the training and validation accuracy curves
epochs = range(1, len(train_accuracy) + 1)

plt.plot(epochs, train_accuracy, 'b', label='Training Accuracy')
plt.plot(epochs, val_accuracy, 'r', label='Validation Accuracy')

plt.title('Training and Validation Accuracy for Transfer Learning')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

In [ ]:
# Saving the trained model
model_transfer_learning.save('trained_model.h5')
model_cnn.save('cnn_trained_model.h5')

In [ ]:
# Evaluate the models
cnn_loss, cnn_accuracy = model_cnn.evaluate(validation_generator)
transfer_loss, transfer_accuracy = model_transfer_learning.evaluate(validation_generator)

print("CNN Model - Loss:", cnn_loss, "Accuracy:", cnn_accuracy)
print("Transfer Learning Model - Loss:", transfer_loss, "Accuracy:", transfer_accuracy)

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Load and preprocess the test image
test_image_path = "sort_sports/2023-07-03_04-16-29_UTC.jpg"
test_image = image.load_img(test_image_path, target_size=(img_height, img_width))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
test_image /= 255.0

In [ ]:
# Make predictions using the CNN model
cnn_predictions = model_cnn.predict(test_image)
cnn_predicted_class = np.argmax(cnn_predictions)
cnn_class_labels = train_generator.class_indices
cnn_predicted_label = list(cnn_class_labels.keys())[list(cnn_class_labels.values()).index(cnn_predicted_class)]

# Make predictions using the transfer learning model
transfer_predictions = model_transfer_learning.predict(test_image)
transfer_predicted_class = np.argmax(transfer_predictions)
transfer_class_labels = train_generator.class_indices
transfer_predicted_label = list(transfer_class_labels.keys())[list(transfer_class_labels.values()).index(transfer_predicted_class)]

In [ ]:
# Print the predictions
print("CNN Predicted Label:", cnn_predicted_label)
print("Transfer Learning Predicted Label:", transfer_predicted_label)

In [ ]:
import json

with open('tags.json', 'r') as json_file:
    hashtags_mapping = json.load(json_file)

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image


# Loading the saved model
loaded_model = tf.keras.models.load_model('trained_model.h5')
# cnn_loaded_model = tf.keras.models.load_model('cnn_trained_model.h5')

def _get_hashtag(label):
    return ','.join([i for i in random.choice(hashtags_mapping[f"{label}"])])


def get_final_hashtags(img_path):
    # Load and preprocess the test image
    test_image_path = img_path
    test_image = image.load_img(test_image_path, target_size=(img_height, img_width))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis=0)
    test_image /= 255.0
    # Make predictions using the CNN model
#     cnn_predictions = cnn_loaded_model.predict(test_image)
#     cnn_predicted_class = np.argmax(cnn_predictions)
#     cnn_class_labels = train_generator.class_indices
#     cnn_predicted_label = list(cnn_class_labels.keys())[list(cnn_class_labels.values()).index(cnn_predicted_class)]

    # Make predictions using the transfer learning model
    transfer_predictions = loaded_model.predict(test_image)
    transfer_predicted_class = np.argmax(transfer_predictions)
    predicted_probability = transfer_predictions[0][transfer_predicted_class]

    if predicted_probability >=.80:
        print(f'predicted with confidence {predicted_probability} moving to get hastags genrator')
        transfer_class_labels = train_generator.class_indices
        transfer_predicted_label = list(transfer_class_labels.keys())[list(transfer_class_labels.values()).index(transfer_predicted_class)]

        res = f"#{transfer_predicted_label},"+f"{_get_hashtag(transfer_predicted_label)}"

    else:
        print(f'predicted with less confidence {predicted_probability} skipping the hastags genrator')

    return res

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
img_path = 'sort_sports/rahul_cricket.jpg'
test_image = Image.open(img_path)

# Display the test image
plt.imshow(test_image)
plt.axis('off')
plt.show()

get_final_hashtags(img_path)

In [ ]:
transfer_predictions = loaded_model.predict(test_image)
transfer_predicted_class = np.argmax(transfer_predictions)
transfer_class_labels = train_generator.class_indices

# Get the class label and its corresponding probability
predicted_label = list(transfer_class_labels.keys())[transfer_predicted_class]
predicted_probability = transfer_predictions[0][transfer_predicted_class]

# Show the predicted class label and its confidence score
print(f"Predicted Class: {predicted_label}")
print(f"Confidence Score: {predicted_probability}")